##패키지설치

In [ ]:
!apt-get update 
!apt-get install g++ openjdk-8-jdk python-dev python3-dev 
!pip3 install JPype1-py3 
!pip3 install fasttext
!git clone https://github.com/facebookresearch/fastText
!cd fastText && make
!pip install numpy

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [1,750 kB]
Ign:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:13 https://developer.download

##DATASET

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# car_inspect_record_info_table
import pandas as pd
import numpy as np
import csv

#inspect_record_info_table
df = pd.read_csv('/content/drive/MyDrive/car_inspect_record_info_table.csv') 

#car_type
df2 = pd.read_csv('/content/drive/MyDrive/car_type.csv')
car_type=df2['type']
car_type=list(car_type)
# for i in range(len(car_type)) :
#   car_type[i]=correction(car_type[i]) #description 칼럼과 띄어쓰기및 맞춤법 동화

#company_name
df2=pd.read_csv('/content/drive/MyDrive/company_name.csv')
company_name=df2['name']
company_name=list(company_name)
company_name+=list(df2['adminid_kor'])
company_name.extend(["차케어","마인디즈","오토큐","티스테이션","태평양물산","스피드메이트","타이어프로","원주","평창","애니카랜드","하이카","프라자","T스테이션","군산","금호타이어","기아오토큐","카포유","다스아우토","신천열린카","파인드라이브","구미아주오토","남양주","화성","고양","마산","제트라인모터스","JJ motors"])
company_name=list(set(company_name)) #중복제거
# # for i in range(len(company_name)) :
# #   company_name[i]=correction(company_name[i]) #description 칼럼과 띄어쓰기및 맞춤법 동화




##텍스트전처리





###특수문자제거,형태소분석,맞춤법교정


In [11]:
#특수문자제거
import re
def sub_special(s):
   return re.sub('[-_=+,#/\?:;,^$.@*\"※~&%ㆍ!└』\\‘|\(\)\[\]\<\>`\'…》]', ' ',s)
    
# 형태소분석
!pip3 install konlpy
from konlpy.tag import Okt
okt=Okt()

def morph(s):
  description=''
  tmp = okt.pos(s, stem=True)  
  for token,pos in tmp: 
    if len(token) >= 1 :
      if pos!='Josa' and pos!='Eomi' and pos!='KoreanParticle': 
        description+=' '+token
  return description


#맞춤법, 띄어쓰기 교정 
!pip install git+https://github.com/ssut/py-hanspell.git # 전처리 - 띄어쓰기/ 맞춤법 검사기
from hanspell import spell_checker  
def correction(s):
  result = spell_checker.check(s)
  result.as_dict()  # dict로 출력
  return result[2]



  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-29tkz_k_
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-29tkz_k_
  Created wheel for py-hanspell: filename=py_hanspell-1.1-cp37-none-any.whl size=4854 sha256=3766ed244d6944f1a8bc00fef64fca7ff12c4d749a233e7e507a33cfbf67be90
  Stored in directory: /tmp/pip-ephem-wheel-cache-87f4tpy8/wheels/0a/25/d1/e5e96476dbb1c318cc26c992dd493394fe42b0c204b3e65588
Successfully built py-hanspell


###특정데이터 클렌징

In [16]:
# 특정데이터클렌징
def delete(s):
  s=re.sub(r'\w{2,10}점',' ',s) #지점명 제거
  s=re.sub(r'(\w+)역|(\w+)출구|(\w+)주차장(\w+)|(\w+)쏘카존|(\w+)카센터|(\w+)주민센터|(\w+)사거리|(\w+)테크|(\w+)구청|(\w+)렌트카',' ',s)
  s=re.sub(r'(\w*)km','',s) #적성거리 제거 165 60 14
  s=re.sub(r'\d{1,10}[원]|\d{2,3}[,]\d{3}[원]|\d{2,3}[,]\d{3}[,]\d{3}[원]|\d{2,3}[,]\d{3}|\d{2,3}[,]\d{3}[,]\d{3}',' ',s)  #가격 제거 
  s=re.sub(r'vat',' ',s) 
  s=re.sub(r'\d{7}|\d{6}|\d{2}[w][-]\d{2,4}|\d{2}[w]\d{2,4}|\d{2}[W][-]\d{2,4}|\d{2}[W]\d{2,4}',' ',s) #장애번호 제거
  s=re.sub('장애번호|장애카드번호|장애카드',' ',s)
  s=re.sub(r'\d{2,4}[00]',' ',s) #가격제거
  s=re.sub('\n|\t',' ',s)
  s=re.sub(r'[ㄱ,ㄴ,ㄷ,ㄹ,ㅁ,ㅇ,ㅎ]',' ',s)
  s=s.replace('더뉴',' ')
  s=s.replace('올뉴',' ')
  s=re.sub(r'[k,K]\d{1}',' ',s) 
  s=re.sub(r'[x,X][ ][D,d]|[x,X][D,d]',' ',s) 
  s=s.replace('쏘카존','')
  s=s.replace('xD','')
  s=re.sub(r'\d{3}[ ]\d{2}[ ]\d{2}|\d{2,4}[-]\d{1,2}[-]\d{1,2}|\d{1,2}[월]\d{1,2}[일]|\d{1,2}[월][ ]\d{1,2}[일]',' ',s)  #날짜제거
  s=re.sub(r'\d{2}[하,허,호]\d{0,4}|\d{4,7}',' ',s) #차량번호 제거
  s=re.sub(r'(\w*)카센타|(\w*)공업사|(\w*)모터스',' ',s) 
  s=re.sub(r'(\w+)자동차|[sm,SM,Sm]d{0,2}',' ',s)
  s=re.sub(r'[R]\d{1,4}[ ]\d{1,4}[ H]\d{1,4}',' ',s)
  s=re.sub(r'[R]\d{1,4}|[RA]\d{1,4}',' ',s)
  return s

stopwords = car_type + company_name
stopwords.sort(reverse=True)
def sub_stopword(s):    #수리지점명, 차종제거
  for word in stopwords:
    s=s.replace(word,'')
  s=s.replace('브크','브레이크')
  return s


# token=[]
# for temp in description:
#   i = temp
#   i = str(i)
#   i = sub_special(i)
#   i = sub_stopword(i)
#   i = delete(i)
#   # i = correction(i)
#   # token.append(i)
#   # lesion.append(token)
#   i=morph(i)
#   token.append(i)


# token=list(set(token)) #중복제거



###전처리, csv 생성

In [ ]:
description=df[['inspect_id','inspect_type','description']]
for i,row in description.iterrows():
  id=description.at[i,'inspect_id']
  inspect=description.at[i,'inspect_type']
  s=description.at[i,'description']
  s = str(s)
  s = sub_special(s)
  s = sub_stopword(s)
  s = delete(s)
  s = correction(s)
  with open('drive/MyDrive/refined_data.csv', 'a+', newline='') as f:  #CSV파일로 저장
    writer = csv.writer(f) 
    writer.writerow([id,inspect,s])
description

In [ ]:
# df=df.dropna()
# description=df[['inspect_type','description']]
# inspect_list=list(set(df['inspect_type']))
# for i in inspect_list :
#   content=description[description['inspect_type']==i]['description']
#   content=list(set(content))
#   token=set()
#   for temp in content:  #클렌징
#     s=temp
#     s = str(s)
#     s = sub_special(s)
#     s = sub_stopword(s)
#     s = delete(s)
#     # s = correction(s)
#     token.add(s)
#   with open('drive/MyDrive/inspect_type.csv', 'a+', newline='') as f:  #CSV파일로 저장
#     writer = csv.writer(f) 
#     writer.writerow([i,str(''.join(list(token)))])


##객체명인식기

In [ ]:
# !git clone https://github.com/eagle705/pytorch-bert-crf-ner.git
# !pip3 install torch torchvision
# !pip3 install pytorch_pretrained_bert>=0.4.0
# !pip3 install mxnet>=1.5.0
# !pip3 install gluonnlp>=0.6.0
# !pip3 install sentencepiece>=0.1.6
# !pip3 install git+https://github.com/kmkurn/pytorch-crf#egg=pytorch_crf
# !pip3 install transformers
# !pip3 install tb-nightly
# !pip3 install future
# import os

# from google.colab import drive
# drive.mount('/content/gdrive')

In [ ]:
# !python pytorch-bert-crf-ner/inference.py
import shutil
shutil.copyfile('/content/drive/MyDrive/best-epoch-12-step-1000-acc-0.960.bin', '/content/sample_data/best-epoch-12-step-1000-acc-0.960.bin')

In [ ]:
!python3 pytorch-bert-crf-ner/inference.py

In [ ]:
list_of_ner_word= [{'word': ' 왕십리역10번출구', 'tag': 'LOC', 'prob': None}, {'word': ' 유현진', 'tag': 'PER', 'prob': None}, {'word': ':239123', 'tag': 'POH', 'prob': None}, {'word': ' 창동역사거리', 'tag': 'LOC', 'prob': None}]

# for i in list_of_ner_word:
#   if i['tag']=='LOC'or'PER'or'ORG'or'DAT'or'TIM'or'DUR'or'MNY'or'NOH' :
#     list_of_ner_word.remove(i)
#     print(list_of_ner_word)
input_text="왕십리역10번출구 지하주차장 김수용 유현진 비용:239123 안녕하세용"
token=set()
for i in list_of_ner_word:
   if i['tag']=='LOC':
     token.add(i['word'])
token



{' 왕십리역10번출구', ' 창동역사거리'}

##빈도수계산(키워드추출)

In [ ]:
!pip3 install krwordrank
from krwordrank.word import KRWordRank
from krwordrank.hangle import normalize

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/inspect_type.csv',names=['inspect_type','content'])
df=df.dropna()
for i in inspect_list:
  contents=df[description['inspect_type']==i]['content']
  contents=list(contents)

  texts = [normalize(text, english=True, number=True) for text in contents]  # 문장내 특수문자 제거하기
  print(texts)
  wordrank_extractor = KRWordRank(
      min_count = 5, # 단어의 최소 출현 빈도수 (그래프 생성 시)
      max_length = 200, # 단어의 최대 길이
      verbose = True
      )
  beta = 0.85    # PageRank의 decaying factor beta
  max_iter = 10
  while(True):
    try:
      keywords, rank, graph = wordrank_extractor.extract(texts, beta, max_iter)
      break
    except:
      print(i)

  for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:30]:
    print('%8s:\t%.4f' % (word, r))


##라이브러리 설치, DATASET

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

##GPU 사용 시
device = torch.device("cuda:0")

bertmodel, vocab = get_pytorch_kobert_model()

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-62f5k4wj
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-62f5k4wj
  Created wheel for kobert: filename=kobert-0.1.2-cp37-none-any.whl size=12708 sha256=1857675db91fcbaf8be9919225679dbee91c4a58a3e3092f165837fb6747c148
  Stored in directory: /tmp/pip-ephem-wheel-cache-h0sylmv3/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert
using cached model
using cached model


In [ ]:
print(vocab)

Vocab(size=8002, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")


In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


##데이터로드

In [ ]:
dataset =lesion
dataset

[]

In [ ]:
# 가져온 내 데이터를 훈련 테스트셋과 테스트 데이터 셋으로 나누기 
from sklearn.model_selection import train_test_split   
#dataset_train, dataset_test = train_test_split(dataset, test_size=0.2, random_state=42)
#print("train shape is:", len(dataset_train))
#print("test shape is:", len(dataset_test))
# 튜토리얼에서 사용하는 데이터 로드 방식
# dataset_train = nlp.data.TSVDataset('/content/drive/MyDrive/Trigger_list.txt', field_indices=[1,2], num_discard_samples=1)
# dataset_test = nlp.data.TSVDataset("/content/drive/MyDrive/Trigger_list.txt", field_indices=[1,2], num_discard_samples=1)
dataset_train = nlp.data.TSVDataset('/content/token.csv')
dataset_test = nlp.data.TSVDataset('/content/token.csv')

FileNotFoundError: ignored

In [ ]:
print(type(dataset_train))
for i in range(len(dataset_train)):
  print(dataset_train[i])

##모델학습

In [ ]:
## Setting parameters
max_len = 128  # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 64
warmup_ratio = 0.1
num_epochs = 180
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]
        self.labels=0

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))



In [ ]:

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=151,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
 
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)


def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

import torch
torch.cuda.empty_cache()

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0

    # train 데이터 학습
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    # test 데이터 학습
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))


    unseen_test = pd.DataFrame([['타이어 파스', 7]], columns = [['질문 내용', '질병명']])
unseen_values = unseen_test.values

test_set = BERTDataset(unseen_values, 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=5)

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  out = model(token_ids, valid_length, segment_ids)
  print(out)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


TypeError: ignored